# Initiation

In [1]:
from pyhive import presto
from pyhive import presto
import pandas as pd
import xgboost as xgb
def sql(query_sentence):
    cursor = presto.connect(protocol='https',
                            host="", #insert your host name/url
                            port= , #insert port
                            username='',
                            password='',
                            session_props={'query_max_stage_count': 500}).cursor()
    cursor.execute(query_sentence)
    return pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import gc

from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import joblib


# intended for use with XGBoost models or similar where preds are the predicted probabilities and dtrain is the training data.
# Its more integrated with XGBoost's internal data handling (dtrain.get_label())
def evalKS(preds, dtrain):
    labels = dtrain.get_label()
    fpr, tpr, threshold = roc_curve(labels, preds)
    KS = max(tpr-fpr)
    return('KS', KS)
    
# This function is more generic and seems to accept labels (true labels) and probs (predicted probabilities) directly as arguments.
def get_ks(labels, probs):
    fpr,tpr,_ = roc_curve(labels,probs)
    w=tpr-fpr
    ks_score=w.max()
    return 'ks',ks_score,1


def get_performance(model,data,fea,labels,title_name):
    nm = max([len(n) for n in title_name])
    
    for i in range(len(title_name)):
        features= data[i][fea]
        probs = model.predict_proba(features)[:,1]
        fpr,tpr,_ = roc_curve(data[i][labels],probs,drop_intermediate=False)
        auc_score = auc(fpr,tpr)
        w=tpr-fpr
        ks_score=w.max()
        ks_x=fpr[w.argmax()]
        ks_y=tpr[w.argmax()]
        

        print(title_name[i],' '*(nm-len(title_name[i])),' features.shape=',features.shape,
        '  auc_score={}'.format(roc_auc_score(data[i][labels],probs).round(5)),
        '  ks_score={}'.format(ks_score.round(5)))


In [2]:
data_1 = sql("""select * from source""")
data_1.rename(columns={'score_test2':'score_1'},inplace=True)
data_1.shape

(261294, 65)

In [5]:
data_2 = sql("""select * from source""")
data_2.rename(columns={'score':'score_2','main_iou_id':'iou_id'},inplace=True)
data_2.shape

(261294, 60)

In [7]:
data_3 = sql("""select * from source""")
data_3.rename(columns={'score':'score_3','main_iou_id':'iou_id'},inplace=True)
data_3.shape

(261294, 56)

In [8]:
data_1 = data_1.replace('', np.nan)
varlist = data_1.iloc[:,15:len(data_1.columns)].columns
for i in list(data_1[varlist].select_dtypes('O')):
    data_1[i] = data_1[i].astype(float)

In [9]:
data_2 = data_2.replace('', np.nan)
varlist = data_2.iloc[:,5:len(data_2.columns)-1].columns
for i in list(data_2[varlist].select_dtypes('O')):
    data_2[i] = data_2[i].astype(float)

In [10]:
data_3 = data_3.replace('', np.nan)
varlist = data_3.iloc[:,5:len(data_3.columns)-1].columns
for i in list(data_3[varlist].select_dtypes('O')):
    data_3[i] = data_3[i].astype(float)

In [11]:
data_3.score_3.value_counts().head()

0.169480    148559
0.132873       329
0.255534       190
0.130754       168
0.157308       130
Name: afi_score, dtype: int64

In [12]:
data_1.score_1.value_counts().head()

0.138776    178972
0.466618       599
0.518462       423
0.310054       412
0.507755       360
Name: pid_score, dtype: int64

In [13]:
pid_list=[]
for i in list(data_1):
    if 'mf' not in i:
        pid_list.append(i)

In [14]:
data_total = data_1[pid_list].merge(
    data_2[['iou_id','score_2']],on='iou_id').merge(
    data_3[['iou_id','score_3']],on='iou_id')
data_total.shape

(261294, 20)

# Sample Division

In [21]:
train_id = sql(""" select * from source """)
train_id.shape

(61748, 1)

In [22]:
valid_id = sql("""select * from source""")
valid_id.shape

(15438, 1)

In [23]:
set(train_id.main_person_uuid).intersection(set(valid_id.main_person_uuid))

set()

In [24]:
train = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(train_id.main_person_uuid))]
valid = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(valid_id.main_person_uuid))]
oot = data_total[~data_total.main_iou_mon.between('2024-04','2024-05')]
train.shape,valid.shape,oot.shape

((96740, 20), (24057, 20), (140497, 20))

In [25]:
96740+24057+140497

261294

# XGB-MO 

In [46]:
var_list = ['score_1','score_2','score_3']
# var_list = ['score_1','score_2']
len(var_list)

3

In [27]:
dtrain = xgb.DMatrix(train[var_list], label=train['flag'])
dvalid = xgb.DMatrix(valid[var_list], label=valid['flag'])

In [ ]:
params_xgb = {
    'booster': 'gbtree',               
    'objective': 'binary:logistic',    
    'disable_default_eval_metric': 1,  
    'gamma': 0,                        
    'max_depth': 3,                    
    'colsample_bytree':1,             
    'subsample': 0.8,                   
    'eta': 0.1,                       
    'seed': 10,                       
    'min_child_weight':200,            
    'tree_method': 'auto'            
    #'max_bin':                        
}

In [ ]:
num_boost_round = 1000                         
evallist = [(dtrain, 'train'),(dvalid, 'valid')]       
evals_result={}
xgb_mod = xgb.train(params_xgb, dtrain, num_boost_round,
                    evallist,feval=evalKS,                   
                    evals_result=evals_result,
                    maximize=True,                          
                    verbose_eval=10,                        
                    early_stopping_rounds=50                
                   )

In [ ]:
def get_performance_v2(model, data, fea, labels, title_name):
    nm = max([len(n) for n in title_name])
    
    for i in range(len(title_name)):
        features = data[i][fea]
        # Use predict to get probabilities for binary classification
        probs = model.predict(xgb.DMatrix(features))
        fpr, tpr, _ = roc_curve(data[i][labels], probs, drop_intermediate=False)
        auc_score = auc(fpr, tpr)
        w = tpr - fpr
        ks_score = w.max()
        ks_x = fpr[w.argmax()]
        ks_y = tpr[w.argmax()]
        
        print(title_name[i], ' ' * (nm - len(title_name[i])), ' features.shape=', features.shape,
              '  auc_score={}'.format(roc_auc_score(data[i][labels], probs).round(5)),
              '  ks_score={}'.format(ks_score.round(5)))

get_performance_v2(
    xgb_mod,
    [train_df_fei, test_df_fei],
    prod_features,
    'isbad',
    ['train', 'valid']
)

# Save model0 

In [ ]:
joblib.dump(xgb_mod,'model0.pkl')

In [ ]:
print('train:',train.shape[0], get_ks(train['flag'], xgb_mod.predict(dtrain))[1].round(4))
print('valid:',valid.shape[0], get_ks(valid['flag'], xgb_mod.predict(dvalid))[1].round(4))
print('oot6:',oot6.shape[0], get_ks(oot6['flag'], xgb_mod.predict(xgb.DMatrix(oot6[var_list])))[1].round(4))
print('oot7:',oot7.shape[0], get_ks(oot7['flag'], xgb_mod.predict(xgb.DMatrix(oot7[var_list])))[1].round(4))

In [ ]:
len(var_list)

In [ ]:
varlist_imp = pd.Series(xgb_mod.get_score(importance_type='total_gain')).sort_values(ascending=False)
print(varlist_imp.shape[0])
varlist_imp

In [ ]:
set(var_list)-set(varlist_imp.index)

In [ ]:
from pysam import Pysam

In [ ]:
var_list

In [ ]:
data = data_total.copy()
data.shape

In [ ]:
data['rh_score'] = xgb_mod.predict(xgb.DMatrix(data[xgb_mod.feature_names]))
data['bins'],binlist = pd.qcut(data['rh_score'],10,duplicates='drop',retbins=True)
data['bins'].value_counts()

In [ ]:
data['rh_score'].value_counts().head(3)

In [ ]:
# pm = Pysam()

# data['seg_v'] = 'other'
# data.loc[(data.iou_id.isin(train.iou_id)), 'seg_v'] = 'dev'
# data.loc[(data.iou_id.isin(valid.iou_id)), 'seg_v'] = 'val'
# data.loc[(data.main_iou_mon.isin(['2024-06','2024-07'])), 'seg_v'] = 'oot'
# data['phat'] = data['rh_score']

# pm.set_data(data)
# pm.config['data_param']['dep_var']='flag'
# #pm.config['reporting_param']['K3']['bins']=list(binlist)

# pm.config['reporting_param']['date_var'] = 'main_iou_mon'
# pm.config['reporting_param']['date_base'] = '2024-04'
# pm.config['reporting_param']['n_top'] = 100
# pm.model = [
#     {'name': '印尼lzdB卡-融合模型', 'model': xgb_mod}
# ]

# pm.standard_report(sheets=['K1', 'K2', 'K3', 'K4', 'K4-1', 'K4-2'])

In [81]:
var_list = ['score_1','score_2','score_3']

In [82]:
train = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(train_id.main_person_uuid))]
valid = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(valid_id.main_person_uuid))]
oot = data_total[~data_total.main_iou_mon.between('2024-04','2024-05')]
train.shape,valid.shape,oot.shape

((96740, 22), (24057, 22), (140497, 22))

In [83]:
dtrain = xgb.DMatrix(train[var_list], label=train['flag'])
dvalid = xgb.DMatrix(valid[var_list], label=valid['flag'])

In [84]:
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.metrics import roc_curve
import numpy as np

prod_features = var_list
final_train_df_exp11 = train
final_test_df_exp11 = valid
final_train_df_exp11['isbad'] = final_train_df_exp11['flag']
final_test_df_exp11['isbad'] = final_test_df_exp11['flag']

# Define the space of hyperparameters to search for XGBoost
xgb_space = {
    'objective': 'binary:logistic',
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),
    'subsample': hp.uniform('subsample', 0.7, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1),
    'gamma': hp.uniform('gamma', 0, 10),
    'max_delta_step': hp.uniform('max_delta_step', 1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0, 15),
    'min_child_weight': hp.uniform('min_child_weight', 1, 100),
    'n_estimators': 1000,
    'random_state': 10
}

# Define the KS calculation function
def get_ks(y_true, y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    ks_statistic = max(tpr - fpr)
    return ks_statistic

# Custom evaluation function for KS
def ks_eval_metric(preds, dtrain):
    labels = dtrain.get_label()
    ks_value = get_ks(labels, preds)
    return 'ks', ks_value

# final_train_df_exp11 = train
# final_train_df_exp11['isbad'] = final_train_df_exp11['flag']
# final_test_df_exp11 = valid
# final_test_df_exp11['isbad'] = final_test_df_exp11['flag']
# prod_features = rh_var

# Define the objective function for XGBoost
def xgb_objective(params):
    clf = xgb.XGBClassifier(**params)
    clf.fit(final_train_df_exp11[prod_features], final_train_df_exp11['isbad'],
            eval_set=[(final_train_df_exp11[prod_features], final_train_df_exp11['isbad']),
                      (final_test_df_exp11[prod_features], final_test_df_exp11['isbad'])],
            eval_metric=ks_eval_metric,  # Use custom KS evaluation
            early_stopping_rounds=50, verbose=20)
    
    # Predict probabilities for the test set
    pred = clf.predict_proba(final_test_df_exp11[prod_features])[:, 1]
    
    # Calculate KS
    ks = get_ks(final_test_df_exp11['isbad'], pred)
    
    print(f"KS: {ks}")
    
    return {'loss': -ks, 'status': STATUS_OK}  # Optimizing for KS by minimizing the negative KS


# Run Hyperopt to find the best hyperparameters
xgb_trials = Trials()
xgb_best = fmin(fn=xgb_objective,
                space=xgb_space,
                algo=tpe.suggest,
                max_evals=20,  
                trials=xgb_trials)

print("Best hyperparameters for XGBoost:", xgb_best)


[0]	validation_0-logloss:0.47120	validation_0-ks:0.28161	validation_1-logloss:0.46364	validation_1-ks:0.28336          
[20]	validation_0-logloss:0.39392	validation_0-ks:0.51862	validation_1-logloss:0.39429	validation_1-ks:0.49016         
[40]	validation_0-logloss:0.36695	validation_0-ks:0.51922	validation_1-logloss:0.37096	validation_1-ks:0.49214         
[50]	validation_0-logloss:0.36140	validation_0-ks:0.51997	validation_1-logloss:0.36664	validation_1-ks:0.49238         
KS: 0.2833617442566201                                                                                                 
[0]	validation_0-logloss:0.47258	validation_0-ks:0.26405	validation_1-logloss:0.46494	validation_1-ks:0.25374          
[20]	validation_0-logloss:0.41132	validation_0-ks:0.51365	validation_1-logloss:0.40949	validation_1-ks:0.48688         
[40]	validation_0-logloss:0.38205	validation_0-ks:0.51359	validation_1-logloss:0.38325	validation_1-ks:0.49062         
[50]	validation_0-logloss:0.37445	valida

In [85]:
def get_ks(preds, dtrain):
    labels = dtrain.get_label()  # Extract true labels
    fpr, tpr, _ = roc_curve(labels, preds)  # Compute ROC curve
    ks_score = (tpr - fpr).max()  # Calculate KS statistic
    return 'ks', ks_score  # Return metric name and value

# def get_ks(pred_probs, dtrain):
#     labels = dtrain.get_label()  # Extract the true labels from DMatrix (i.e., the 'isbad_15d' column)
#     fpr, tpr, _ = roc_curve(labels, pred_probs)  # Compute FPR and TPR
#     ks_score = np.max(tpr - fpr)  # KS statistic is max(TPR - FPR)
#     return 'ks', ks_score, True 
# evals = [(dtrain, 'train'), (dtest, 'eval')]
# def get_ks(labels, probs):
#     fpr,tpr,_ = roc_curve(labels,probs)
#     w=tpr-fpr
#     ks_score=w.max()
#     return 'ks',ks_score,1
train_df_fei = train
# final_train_df_fei = train
# final_train_df_fei['isbad'] = final_train_df_fei['flag']
test_df_fei = valid
# test_df_fei['isbad'] = test_df_fei['flag']
final_features = var_list

xgb_clf = xgb.XGBClassifier(**xgb_best)
xgb_clf.fit(train_df_fei[final_features], train_df_fei['isbad'],
            eval_set=[(train_df_fei[final_features], train_df_fei['isbad']),
                      (test_df_fei[final_features], test_df_fei['isbad'])],
            eval_metric=get_ks,  # Custom KS function
            early_stopping_rounds=200, verbose=20
           )

get_performance(xgb_clf,
                [train_df_fei,test_df_fei],
                final_features,
                'isbad',
                ['train','valid']
               )

[0]	validation_0-logloss:0.44083	validation_0-ks:0.49478	validation_1-logloss:0.43605	validation_1-ks:0.46329
[20]	validation_0-logloss:0.35338	validation_0-ks:0.51829	validation_1-logloss:0.36086	validation_1-ks:0.49395
[40]	validation_0-logloss:0.35117	validation_0-ks:0.51769	validation_1-logloss:0.36141	validation_1-ks:0.49307
[60]	validation_0-logloss:0.35070	validation_0-ks:0.51876	validation_1-logloss:0.36180	validation_1-ks:0.49364
[80]	validation_0-logloss:0.35047	validation_0-ks:0.51900	validation_1-logloss:0.36195	validation_1-ks:0.49301
[99]	validation_0-logloss:0.35025	validation_0-ks:0.52063	validation_1-logloss:0.36190	validation_1-ks:0.49302
train   features.shape= (96740, 3)   auc_score=0.80717   ks_score=0.49477
valid   features.shape= (24057, 3)   auc_score=0.78962   ks_score=0.46329


In [86]:
joblib.dump(xgb_mod,'CM_XGB.pkl')

['CM_XGB.pkl']

In [87]:
cm_xgb = joblib.load('CM_XGB.pkl')

In [88]:
train_lr = train.copy()  # only dev data
train = train.copy()
rh_var = var_list
train_lr[rh_var] = train_lr[rh_var].apply(pd.to_numeric, errors='coerce')
train_lr[rh_var].replace(to_replace=[None, "None", np.nan], value=-999, inplace=True)

base_data = train_lr[rh_var]
base_data['isbad'] = train_lr.flag

import scorecardpy as sc
bins_df = sc.woebin(train, y='isbad', x = rh_var)
# Applying WoE Binning to the dataset
df_binned = sc.woebin_ply(base_data, bins_df)

[INFO] creating woe binning ...
[INFO] converting into woe values ...


In [89]:
# Create model
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE #for feature selection
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.metrics import (f1_score,classification_report,
                             roc_auc_score,confusion_matrix,roc_curve,auc) # evaluatin metrics
from sklearn.model_selection import KFold,RandomizedSearchCV ,cross_val_score,RepeatedStratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_LR = LogisticRegression()
model_LR.fit( df_binned.drop(columns = ['isbad']), df_binned.isbad )

LogisticRegression()

In [90]:
# Prepare Test Data
# final_features = rh_var
valid_lr = valid.copy()
final_features = var_list
valid_lr[final_features] = valid_lr[final_features].apply(pd.to_numeric, errors='coerce')
import numpy as np

valid_lr[final_features].replace(to_replace=[None, "None", np.nan], value=-999, inplace=True)

base_data_test = valid_lr[final_features]
base_data_test['isbad'] = valid_lr.flag

df_binned_test = sc.woebin_ply(base_data_test, bins_df)

[INFO] converting into woe values ...


In [91]:
get_performance(model_LR,
                [df_binned, df_binned_test],
                df_binned.drop(columns = 'isbad').columns,
                'isbad',
                ['train','valid']
               )

train   features.shape= (96740, 3)   auc_score=0.8197   ks_score=0.5018
valid   features.shape= (24057, 3)   auc_score=0.80254   ks_score=0.4864


In [92]:
joblib.dump(model_LR, 'CM_LR.pkl')

['CM_LR.pkl']

In [93]:
CM_LR = joblib.load('CM_LR.pkl')

In [94]:
# Define the space of hyperparameters to search for LightGBM
lgb_space = {
    'objective': 'binary',
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),
    'subsample': hp.uniform('subsample', 0.7, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1),
    'min_child_samples': hp.choice('min_child_samples', [20, 50, 100, 200, 500]),
    'max_bin': hp.choice('lgb_max_bin', [255, 300, 350, 400]),
    'min_split_gain': hp.uniform('min_split_gain', 0, 15),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0, 15),
    'n_estimators': 1000,
    'random_state': 10
}
def get_ks(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    ks_score = max(tpr - fpr)
    return 'ks', ks_score, True  # The third value should be True, indicating higher is better

# train_df_fei = train
# train_df_fei['isbad'] = train_df_fei['flag']
# test_df_fei = valid
# test_df_fei['isbad'] = test_df_fei['flag']
# final_features = rh_var

# Define the objective function for LightGBM
def lgb_objective(params):
    clf = lgb.LGBMClassifier(**params)
    clf.fit(train_df_fei[final_features], train_df_fei['isbad'],
            eval_set=[(train_df_fei[final_features], train_df_fei['isbad']),
                      (test_df_fei[final_features], test_df_fei['isbad'])],
             eval_metric=get_ks,early_stopping_rounds=50, verbose=20
           )
    pred = clf.predict_proba(final_test_df_exp11[prod_features])[:, 1]
    auc = roc_auc_score(final_test_df_exp11['isbad'], pred)
    return {'loss': -auc, 'status': STATUS_OK}

# Continue from the LightGBM optimization
lgb_trials = Trials()
lgb_best = fmin(fn=lgb_objective,
                space=lgb_space,
                algo=tpe.suggest,
                max_evals=100,  # Number of iterations; adjust as needed
                trials=lgb_trials)

print("Best hyperparameters for LightGBM:", lgb_best)

[20]	valid_0's binary_logloss: 0.350003	valid_0's ks: 0.526171	valid_1's binary_logloss: 0.360907	valid_1's ks: 0.488273
[40]	valid_0's binary_logloss: 0.349345	valid_0's ks: 0.525885	valid_1's binary_logloss: 0.361299	valid_1's ks: 0.488409
[60]	valid_0's binary_logloss: 0.349345	valid_0's ks: 0.525885	valid_1's binary_logloss: 0.361299	valid_1's ks: 0.488409
[20]	valid_0's binary_logloss: 0.361928	valid_0's ks: 0.517274	valid_1's binary_logloss: 0.368186	valid_1's ks: 0.487807
[40]	valid_0's binary_logloss: 0.352339	valid_0's ks: 0.520534	valid_1's binary_logloss: 0.362244	valid_1's ks: 0.488758
[60]	valid_0's binary_logloss: 0.350956	valid_0's ks: 0.520807	valid_1's binary_logloss: 0.361953	valid_1's ks: 0.489527
[80]	valid_0's binary_logloss: 0.350111	valid_0's ks: 0.522371	valid_1's binary_logloss: 0.361744	valid_1's ks: 0.490671
[100]	valid_0's binary_logloss: 0.349973	valid_0's ks: 0.52297	valid_1's binary_logloss: 0.36163	valid_1's ks: 0.49095 
[120]	valid_0's binary_logloss: 0

In [95]:
final_train_df_fei = train
# final_train_df_fei['isbad'] = final_train_df_fei['flag']
# test_df_fei = valid
# test_df_fei['isbad'] = test_df_fei['flag']
# final_features = rh_var

def get_ks(labels, probs):
    fpr,tpr,_ = roc_curve(labels,probs)
    w=tpr-fpr
    ks_score=w.max()
    return 'ks',ks_score,1

lgb_clf_challenger = lgb.LGBMClassifier(**lgb_best)
lgb_clf_challenger.fit(final_train_df_fei[final_features],final_train_df_fei['isbad'], 
            eval_set=[(train_df_fei[final_features], train_df_fei['isbad']),(test_df_fei[final_features],test_df_fei['isbad'])],
            eval_names=['train','valid'],
            eval_metric=get_ks
            ,early_stopping_rounds=50,verbose=20
           )

get_performance(lgb_clf_challenger,
                [train_df_fei,
                 test_df_fei
                ],
                prod_features,
                'isbad',
                ['train','valid']
               )

[LightGBM] [Warning] Unknown parameter: lgb_max_bin
[20]	train's binary_logloss: 0.358673	train's ks: 0.515593	valid's binary_logloss: 0.364153	valid's ks: 0.49072
[40]	train's binary_logloss: 0.352237	train's ks: 0.520672	valid's binary_logloss: 0.361477	valid's ks: 0.488632
[60]	train's binary_logloss: 0.351201	train's ks: 0.521215	valid's binary_logloss: 0.361844	valid's ks: 0.487267
train   features.shape= (96740, 3)   auc_score=0.83461   ks_score=0.51737
valid   features.shape= (24057, 3)   auc_score=0.81558   ks_score=0.49213


In [ ]:
# print("Training data shape:", train_df_fei[final_features].shape)
# print("Features used:", final_features)
# print("Labels distribution:", train_df_fei['isbad'].value_counts())


In [97]:
joblib.dump(lgb_clf_challenger, 'CM_LGBM.pkl')
CM_LGBM = joblib.load('CM_LGBM.pkl')

In [ ]:
# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# from sklearn.metrics import roc_auc_score, roc_curve
# import catboost as cb

# # Define the space of hyperparameters to search for CatBoost
# cat_space = {
#     'depth': hp.choice('depth', [4, 6, 8, 10]),
#     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
#     'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
#     'subsample': hp.uniform('subsample', 0.7, 1),
#     'colsample_bylevel': hp.uniform('colsample_bylevel', 0.7, 1),
#     'min_data_in_leaf': hp.choice('min_data_in_leaf', [20, 50, 100, 200, 500]),
#     'iterations': 1000,  # Fixed number of iterations
#     'random_seed': 10,
#     'eval_metric': 'AUC',
#     'loss_function': 'Logloss'
# }

# # Define the Kolmogorov-Smirnov statistic for evaluation
# def get_ks(y_true, y_pred):
#     fpr, tpr, _ = roc_curve(y_true, y_pred)
#     ks_score = max(tpr - fpr)
#     return 'ks', ks_score, True  # The third value should be True, indicating higher is better

# # Replace these with your actual datasets and features
# # train_df_fei = train
# # train_df_fei['isbad'] = train_df_fei['flag']
# # test_df_fei = valid
# # test_df_fei['isbad'] = test_df_fei['flag']
# # final_features = rh_var

# # Define the objective function for CatBoost
# def cat_objective(params):
#     clf = cb.CatBoostClassifier(**params, verbose=0)
#     clf.fit(
#         train_df_fei[final_features], train_df_fei['isbad'],
#         eval_set=[(train_df_fei[final_features], train_df_fei['isbad']),
#                   (test_df_fei[final_features], test_df_fei['isbad'])],
#         early_stopping_rounds=50
#     )
#     pred = clf.predict_proba(final_test_df_exp11[prod_features])[:, 1]
#     auc = roc_auc_score(final_test_df_exp11['isbad'], pred)
#     return {'loss': -auc, 'status': STATUS_OK}

# # Perform optimization
# cat_trials = Trials()
# cat_best = fmin(
#     fn=cat_objective,
#     space=cat_space,
#     algo=tpe.suggest,
#     max_evals=100,  # Number of iterations; adjust as needed
#     trials=cat_trials
# )

# print("Best hyperparameters for CatBoost:", cat_best)


In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import roc_curve

# # Define the KS metric for evaluation
# def get_ks(labels, probs):
#     fpr, tpr, _ = roc_curve(labels, probs)
#     w = tpr - fpr
#     ks_score = w.max()
#     return 'ks', ks_score, True  # Third value should be True, indicating higher is better

# # Initialize CatBoostClassifier with best parameters
# cat_clf_challenger = CatBoostClassifier(
#     **cat_best,  # Replace with the best parameters found from hyperparameter tuning
#     eval_metric='AUC',  # Base evaluation metric
#     custom_metric='Logloss',  # Add custom metrics if desired
#     verbose=20  # Verbose interval for logging
# )

# # Train the CatBoost model
# cat_clf_challenger.fit(
#     final_train_df_fei[final_features], final_train_df_fei['isbad'],
#     eval_set=[
#         (train_df_fei[final_features], train_df_fei['isbad']),
#         (test_df_fei[final_features], test_df_fei['isbad'])
#     ],
#     early_stopping_rounds=50,
#     use_best_model=True
# )

# # Evaluate performance on both train and validation sets
# get_performance(
#     cat_clf_challenger,
#     [train_df_fei, test_df_fei],
#     prod_features,
#     'isbad',
#     ['train', 'valid']
# )


In [ ]:
# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score, roc_curve
# import numpy as np

# # Define the space of hyperparameters for Random Forest
# rf_space = {
#     'n_estimators': hp.choice('n_estimators', [100, 200, 500, 1000]),  # Number of trees
#     'max_depth': hp.choice('max_depth', [5, 10, 15, 20, None]),  # Tree depth
#     'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.2),  # Min samples to split a node
#     'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.2),  # Min samples at a leaf
#     'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),  # Max features to consider for splitting
#     'bootstrap': hp.choice('bootstrap', [True, False]),  # Bootstrap sampling
#     'random_state': 10  # Fixed random seed
# }

# # Kolmogorov-Smirnov statistic for evaluation
# def get_ks(y_true, y_pred):
#     fpr, tpr, _ = roc_curve(y_true, y_pred)
#     ks_score = max(tpr - fpr)
#     return 'ks', ks_score, True  # Higher is better

# # Define the objective function for Random Forest
# def rf_objective(params):
#     clf = RandomForestClassifier(**params)
#     clf.fit(train_df_fei[final_features], train_df_fei['isbad'])
    
#     # Predict probabilities for the validation set
#     pred = clf.predict_proba(test_df_fei[final_features])[:, 1]
#     auc = roc_auc_score(test_df_fei['isbad'], pred)
    
#     return {'loss': -auc, 'status': STATUS_OK}

# # Perform optimization
# rf_trials = Trials()
# rf_best = fmin(
#     fn=rf_objective,
#     space=rf_space,
#     algo=tpe.suggest,
#     max_evals=100,  # Number of iterations
#     trials=rf_trials
# )

# print("Best hyperparameters for Random Forest:", rf_best)


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score, roc_curve

# # Define the KS metric for evaluation
# def get_ks(labels, probs):
#     fpr, tpr, _ = roc_curve(labels, probs)
#     w = tpr - fpr
#     ks_score = w.max()
#     return 'ks', ks_score, True  # Higher is better

# # Initialize RandomForestClassifier with the best parameters
# rf_clf_challenger = RandomForestClassifier(
#     n_estimators=rf_best_fixed['n_estimators'],
#     max_depth=rf_best_fixed['max_depth'],
#     max_features=rf_best_fixed['max_features'],
#     min_samples_leaf=rf_best_fixed['min_samples_leaf'],
#     min_samples_split=rf_best_fixed['min_samples_split'],
#     bootstrap=rf_best_fixed['bootstrap'],
#     random_state=10,
#     verbose=1
# )


# # Train the Random Forest model
# rf_clf_challenger.fit(
#     final_train_df_fei[final_features], 
#     final_train_df_fei['isbad']
# )

# # Function to evaluate performance on multiple datasets
# def get_performance(model, datasets, features, label, dataset_names):
#     for df, name in zip(datasets, dataset_names):
#         probs = model.predict_proba(df[features])[:, 1]  # Get probabilities for the positive class
#         auc = roc_auc_score(df[label], probs)
#         ks_metric = get_ks(df[label], probs)
#         print(f"{name} Set: AUC = {auc:.4f}, KS = {ks_metric[1]:.4f}")

# # Evaluate performance on train and validation sets
# get_performance(
#     rf_clf_challenger,
#     [train_df_fei, test_df_fei],
#     prod_features,
#     'isbad',
#     ['train', 'valid']
# )

In [ ]:
# rf_best_fixed = {
#     'bootstrap': bool(rf_best['bootstrap']),
#     'max_depth': rf_best['max_depth'],
#     'max_features': rf_best['max_features'],
#     'min_samples_leaf': max(1, int(rf_best['min_samples_leaf'] * len(final_train_df_fei))),  # Convert to integer if proportion
#     'min_samples_split': max(2, int(rf_best['min_samples_split'] * len(final_train_df_fei))),  # Convert to integer if proportion
#     'n_estimators': max(10, rf_best['n_estimators'])  # Ensure a minimum number of trees
# }


In [ ]:
# print(rf_best)


In [41]:
# with open('afi_lzd_order_11009.bin', 'rb') as b:
#     model = joblib.load(b)

In [43]:
model = joblib.load('afi_lzd_order_11009.bin')

In [45]:
model.feature_names

['pid_score', 'lzd_score', 'afi_score']

In [54]:
df = data_total
var_list = ['score_1', 'score_2', 'score_3']
dmat = xgb.DMatrix(data_total[var_list])
df['probability'] = model.predict(dmat)

In [55]:
df_dev = data_total
# df[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(train_id.main_person_uuid))]
df_dev['score_band'] , bins = pd.qcut(df_dev['probability'], q=10, duplicates = 'drop', retbins = True)
df['score_band'] = pd.cut(df['probability'], bins=bins, include_lowest=True)
k1 = df

In [116]:
k2 = k1
X = k2[['score_1', 'score_2', 'score_3']]
k2['CM_XGB'] = cm_xgb.predict(xgb.DMatrix(X))  # XGBoost expects a DMatrix
# k2['CM_LGBM'] = CM_LGBM.predict(X)
k2['CM_LGBM'] = CM_LGBM.predict_proba(X)[:, 1] 
# k2['CM_LR'] = CM_LR.predict(X)

,iou_id,person_uuid,afi_uid,pay_off,create_time,main_iou_date,main_iou_mon,first_month,first_date,epd,...,flag,has_fea,pid_score,lzd_score,afi_score,probability,score_band,CM_XGB,CM_LGBM,CM_LR
0,643466616110452740,81ddfe194cbeab2d2f6bb2854a9c70a4,428344196417949697,17,1718031997542,2024-06-10,2024-06,2024-07,2024-07-02,86,...,0.0,0.0,0.138776,0.027842,0.169480,0.014414,"(0.00784, 0.016]",0.015358,0.031089,0.0
1,661378072713048068,995f91e81ef87bd6c9557e7b59a9c1ec,501277375742967809,5,1722302421590,2024-07-30,2024-07,2024-09,2024-09-02,24,...,0.0,0.0,0.138776,0.232829,0.169480,0.179655,"(0.143, 0.212]",0.175345,0.186723,0.0
2,629998521488691204,3a393c4a5a7a4465b246cc6f64d019f7,176926906363875328,38,1714820953939,2024-05-04,2024-05,2024-06,2024-06-02,116,...,0.0,1.0,0.240593,0.095031,0.125814,0.097745,"(0.0643, 0.0996]",0.099974,0.114089,0.0
3,658250264008597508,997e5a44a8b84a80debd71e2aedb5641,195355048820174848,40,1721556694014,2024-07-21,2024-07,2024-09,2024-09-02,24,...,0.0,1.0,0.048990,0.066583,0.105287,0.020802,"(0.016, 0.0323]",0.018601,0.046922,0.0
4,648668597204959238,b870430617be09d2bb02584105fd64eb,232010306901221377,29,1719272246495,2024-06-25,2024-06,2024-08,2024-08-02,55,...,0.0,1.0,0.250096,0.103752,0.152608,0.103080,"(0.0996, 0.143]",0.099320,0.118636,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261289,649605689297022979,82509ae250571290ffca1c82a63875aa,177333695832064000,38,1719495667014,2024-06-27,2024-06,2024-08,2024-08-02,55,...,0.0,0.0,0.138776,0.036891,0.136040,0.016971,"(0.016, 0.0323]",0.018622,0.031089,0.0
261290,618932237757411332,708d284c556397172eb0b4e1fa0901ee,235443926181584898,34,1712182546205,2024-04-04,2024-04,2024-05,2024-05-02,147,...,0.0,0.0,0.138776,0.093030,0.169480,0.061463,"(0.0464, 0.0643]",0.062116,0.070319,0.0
261291,644224369003208707,9e384d9dbd9ce6f07676cc0a1392acb4,428234887176626177,5,1718212659913,2024-06-13,2024-06,2024-07,2024-07-02,86,...,0.0,1.0,0.577554,0.074080,0.129995,0.257184,"(0.212, 0.315]",0.297083,0.174399,0.0
261292,653470226192572419,007e18b03d225ba8de74a3fd33d77012,210189063704158208,40,1720417044125,2024-07-08,2024-07,2024-08,2024-08-02,55,...,0.0,0.0,0.138776,0.062718,0.169480,0.041351,"(0.0323, 0.0464]",0.038941,0.050451,0.0


In [118]:
# Assuming k2 contains the features ['score_1', 'score_2', 'score_3'] to predict the target
# Preprocess k2 data (similar to training and validation data)
k2[rh_var] = k2[rh_var].apply(pd.to_numeric, errors='coerce')  # Convert to numeric
k2[rh_var].replace(to_replace=[None, "None", np.nan], value=-999, inplace=True)  # Handle missing values

# Apply WoE binning to k2 data (same bins as the training set)
base_data_k2 = k2[rh_var]
df_binned_k2 = sc.woebin_ply(base_data_k2, bins_df)

# Generate predictions using the Logistic Regression model
# k2['CM_LR'] = model_LR.predict(df_binned_k2)  # Use the same features, excluding the target column
k2['CM_LR'] =  model_LR.predict_proba(df_binned_k2)[:, 1]
# Optionally, use probabilities instead of binary predictions:
# k2['CM_LR_prob'] = model_LR.predict_proba(df_binned_k2.drop(columns=['isbad']))[:, 1]  # For probabilities

# Check the updated k2 DataFrame with predictions
print(k2.head())

[INFO] converting into woe values ...
               iou_id                       person_uuid             afi_uid  \
0  643466616110452740  81ddfe194cbeab2d2f6bb2854a9c70a4  428344196417949697   
1  661378072713048068  995f91e81ef87bd6c9557e7b59a9c1ec  501277375742967809   
2  629998521488691204  3a393c4a5a7a4465b246cc6f64d019f7  176926906363875328   
3  658250264008597508  997e5a44a8b84a80debd71e2aedb5641  195355048820174848   
4  648668597204959238  b870430617be09d2bb02584105fd64eb  232010306901221377   

  pay_off    create_time main_iou_date main_iou_mon first_month  first_date  \
0      17  1718031997542    2024-06-10      2024-06     2024-07  2024-07-02   
1       5  1722302421590    2024-07-30      2024-07     2024-09  2024-09-02   
2      38  1714820953939    2024-05-04      2024-05     2024-06  2024-06-02   
3      40  1721556694014    2024-07-21      2024-07     2024-09  2024-09-02   
4      29  1719272246495    2024-06-25      2024-06     2024-08  2024-08-02   

   epd  ... 

In [57]:
# train = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(train_id.main_person_uuid))]
# valid = data_total[(data_total.main_iou_mon.between('2024-04','2024-05'))&(data_total.person_uuid.isin(valid_id.main_person_uuid))]
# oot = data_total[~data_total.main_iou_mon.between('2024-04','2024-05')]
# train.shape,valid.shape,oot.shape

In [67]:
import pandas as pd
import numpy as np

filtered_df = k1[(k1.main_iou_mon.between('2024-04','2024-05'))&(k1.person_uuid.isin(train_id.main_person_uuid))]
# filtered_df = k1[(k1.main_iou_mon.between('2024-04','2024-05'))&(k1.person_uuid.isin(valid_id.main_person_uuid))]
# filtered_df = k1[~k1.main_iou_mon.between('2024-04','2024-05')]
filtered_df['y_label'] = filtered_df['flag']

def calculate_performance_oot(df):
    # Aggregate data by score band
    performance = df.groupby('score_band').agg(
        Good=('y_label', lambda x: (x == 0).sum()),
        Bad=('y_label', lambda x: (x == 1).sum()),
        Total=('y_label', 'size')
    )
    
    # Calculate Total % for each score band
    performance['Total %'] = performance['Total'] / performance['Total'].sum() * 100
    
    # Calculate Cumulative distributions for KS calculation
    performance['Cumulative Good'] = performance['Good'].cumsum()
    performance['Cumulative Bad'] = performance['Bad'].cumsum()
    performance['Bad %'] = performance['Bad'] / performance['Total']*100
    performance['Cumulative Good %'] = performance['Cumulative Good'] / performance['Good'].sum()
    performance['Cumulative Bad %'] = performance['Cumulative Bad'] / performance['Bad'].sum()
    performance['KS'] = (performance['Cumulative Good %'] - performance['Cumulative Bad %']).abs()

    
    return performance

# Calculate and print performance for the ootreal data
performance_oot = calculate_performance_oot(filtered_df)
print("Performance Metrics for OOT Data:")
performance_oot


Performance Metrics for OOT Data:


,Good,Bad,Total,Total %,Cumulative Good,Cumulative Bad,Bad %,Cumulative Good %,Cumulative Bad %,KS
score_band,,,,,,,,,,
"(0.00784, 0.016]",10095,117,10212,10.556130,10095,117,1.145711,0.127236,0.006725,0.120511
"(0.016, 0.0323]",8985,212,9197,9.506926,19080,329,2.305099,0.240481,0.018909,0.221572
"(0.0323, 0.0464]",9247,366,9613,9.936944,28327,695,3.807344,0.357029,0.039945,0.317084
"(0.0464, 0.0643]",9178,496,9674,10.000000,37505,1191,5.127145,0.472706,0.068452,0.404254
"(0.0643, 0.0996]",9425,838,10263,10.608848,46930,2029,8.165254,0.591497,0.116616,0.474882
"(0.0996, 0.143]",8333,1171,9504,9.824271,55263,3200,12.321128,0.696525,0.183919,0.512607
"(0.143, 0.212]",8083,1738,9821,10.151954,63346,4938,17.696772,0.798402,0.283809,0.514592
"(0.212, 0.315]",6648,2460,9108,9.414927,69994,7398,27.009223,0.882192,0.425197,0.456995
"(0.315, 0.484]",5980,3884,9864,10.196403,75974,11282,39.375507,0.957563,0.648428,0.309135


In [131]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve

# Filter the DataFrame
df = k1
# df = train

# Grouping data by 'main_apply_mon' and calculating the required metrics
summary = df.groupby('main_iou_mon').agg(
    total_count=('flag', 'size'),
    bad=('flag', lambda x: (x == 1).sum()),
).reset_index()

# Calculating the bad rate
summary['bad_rate'] = summary['bad'] / summary['total_count']

# Calculate overall bad rate in the population
overall_bad_rate = df['flag'].mean()  # Mean gives the overall bad rate

# Calculate lift
summary['lift'] = summary['bad_rate'] / overall_bad_rate

# Function to calculate AUC for each month
def calculate_auc(month_data):
    if len(month_data) > 0 and month_data['flag'].nunique() > 1:  # Check for both classes
        return roc_auc_score(month_data['flag'], month_data['probability'])
    return None  # Return None if only one class is present

summary['auc'] = df.groupby('main_iou_mon').apply(calculate_auc).values

# Function to calculate KS statistic for each month
# def calculate_ks(month_data):
#     if len(month_data) > 0:
#         fpr, tpr, _ = roc_curve(month_data['flag'], month_data['probability'])
#         ks_statistic = max(tpr - fpr)
#         return ks_statistic
#     return None  # Return None if no data
def calculate_ks(month_data, probability_column):
    if len(month_data) > 0:
        fpr, tpr, _ = roc_curve(month_data['flag'], month_data[probability_column])
        ks_statistic = max(tpr - fpr)
        return ks_statistic
    return None 

# summary['ks'] = df.groupby('main_iou_mon').apply(calculate_ks).values
summary = df.groupby('main_iou_mon').apply(lambda group: calculate_ks(group, 'CM_LGBM')).reset_index(name='ks')

# Display the summary table
summary

,main_iou_mon,ks
0,2024-04,0.505636
1,2024-05,0.517928
2,2024-06,0.446709
3,2024-07,0.459715


In [119]:
k1.columns

Index(['iou_id', 'person_uuid', 'afi_uid', 'pay_off', 'create_time',
       'main_iou_date', 'main_iou_mon', 'first_month', 'first_date', 'epd',
       'nd1', 'nd15', 'main_iou_id', 'main_person_uuid', 'main_iou_time',
       'flag', 'has_fea', 'pid_score', 'lzd_score', 'afi_score', 'probability',
       'score_band', 'CM_XGB', 'CM_LGBM', 'CM_LR'],
      dtype='object')

In [75]:
k1.nd15.value_counts()

0    214837
1     46457
Name: nd15, dtype: int64